# Этап 1. Подготовка данных

Загрузим данные

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("data/dataset1.csv")

In [3]:
dataset.head()

,Unnamed: 0,PRODUCT,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,0,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Что особенно ценно - при профилактике принтера...,23.10.16 22:26:00,5.0,"хорошая скорость печати, двусторонняя печать, ...","Рядом ""на стол"" не поставишь, место много зани...",1.0,2.0,0.0
1,1,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Он у нас три месяца в офисе стоит уже - большо...,17.04.16 11:17:42,5.0,NaN,NaN,NaN,0.0,2.0
2,2,30024725,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,"Просто небо и земля с тем принтером, который у...",15.04.16 20:22:47,4.0,NaN,NaN,NaN,0.0,1.0
3,3,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,С такой вроде бы навороченной техникой все смо...,У нас в кабинете недавно его поставили. Первое...,18.04.16 20:01:57,5.0,NaN,NaN,NaN,0.0,0.0
4,4,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Очень многофункциональная штука. Есть все нео...,23.04.16 12:03:39,5.0,NaN,NaN,NaN,1.0,0.0


## Предобработка текстов

Определим функцию, применяющую к тексту следующие преобразования:

- использование нижнего регистра
- удаление знаков препинания (потребуется в следующих шагах)
- токенизация
- добавление POS-тегов (потребуются при фильтрации текстов)

In [4]:
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize
from functools import lru_cache


morph = MorphAnalyzer()


@lru_cache(20000)
def morph_parse(token):
    return morph.parse(token)


def text_conversion(text):
    tokens = [token for token in word_tokenize(text.lower()) if len(token) > 0]
    result = []
    for token in tokens:
        morph = morph_parse(token)
        if len(morph) > 0:
            if morph[0].tag.POS is not None:
                result.append(morph[0])
    return " ".join(map(lambda morph: "{0}_{1}".format(morph.word, morph.tag.POS), result))


def texts_conversion(texts, message_each=100):
    result = []
    for i, text in enumerate(texts):
        if i % message_each == 0:
            print("Converting text", i + 1)
        result.append(text_conversion(text))
    return result

Проверим функцию над коллекцией из текстов первой пары отзывов

In [5]:
texts_conversion(dataset.loc[0:1, "TEXT"])

Converting text 1


['что_CONJ особенно_ADVB ценно_ADJS при_PREP профилактике_NOUN принтера_NOUN можно_PRED вычистить_INFN блок_NOUN печати_NOUN от_PREP бумажной_ADJF пыли_NOUN все_ADJF запчасти_NOUN снимаются_VERB каждую_ADJF можно_PRED отдельно_ADVB почистить_INFN щёткой_NOUN и_CONJ пропылесосить_INFN и_CONJ будет_VERB как_CONJ новая_ADJF меньше_COMP пыли_NOUN меньше_COMP еёдавления_NOUN на_PREP фотобарабан_NOUN меньше_COMP давления_NOUN на_PREP фотобарабан_NOUN меньше_COMP его_NPRO стирание_NOUN и_CONJ соответственно_ADVB больше_COMP срок_NOUN службы_NOUN фотовала_NOUN у_PREP нас_NPRO в_PREP организации_NOUN фотовал_NOUN прошёл_VERB двойной_ADJF срок_NOUN службы_NOUN бумагау_NOUN использовыали_VERB отечественную_ADJF ксероксную_ADJF просто_PRCL чистили_VERB ругулярно_ADJS и_CONJ ещё_ADVB один_ADJF мент_NOUN для_PREP повышения_NOUN срока_NOUN службы_NOUN аппарата_NOUN задействуйте_VERB в_PREP нем_ADJS все_ADJF лотки_NOUN для_PREP бумаги_NOUN если_CONJ одновременно_ADVB используете_VERB лотка_NOUN с_PREP

Заполним и модифицируем текстовые поля

In [6]:
dataset["BENEFITS_FILLED"] = dataset["BENEFITS"].fillna("")
dataset["DRAWBACKS_FILLED"] = dataset["DRAWBACKS"].fillna("")
dataset["TEXT_FILLED"] = dataset["TEXT"].fillna("")

In [7]:
dataset["TEXT_CONVERTED"] = texts_conversion(dataset["TEXT_FILLED"])
dataset["BENEFITS_CONVERTED"] = texts_conversion(dataset["BENEFITS_FILLED"])
dataset["DRAWBACKS_CONVERTED"] = texts_conversion(dataset["DRAWBACKS_FILLED"])

Converting text 1
Converting text 101
Converting text 201
Converting text 301
Converting text 401
Converting text 501
Converting text 601
Converting text 701
Converting text 801
Converting text 901
Converting text 1001
Converting text 1101
Converting text 1201
Converting text 1301
Converting text 1401
Converting text 1501
Converting text 1601
Converting text 1701
Converting text 1801
Converting text 1901
Converting text 2001
Converting text 2101
Converting text 2201
Converting text 2301
Converting text 2401
Converting text 2501
Converting text 2601
Converting text 2701
Converting text 2801
Converting text 2901
Converting text 3001
Converting text 3101
Converting text 3201
Converting text 3301
Converting text 3401
Converting text 3501
Converting text 3601
Converting text 3701
Converting text 3801
Converting text 3901
Converting text 4001
Converting text 4101
Converting text 4201
Converting text 4301
Converting text 4401
Converting text 4501
Converting text 4601
Converting text 4701
Conv

Converting text 37801
Converting text 37901
Converting text 38001
Converting text 38101
Converting text 38201
Converting text 38301
Converting text 38401
Converting text 38501
Converting text 38601
Converting text 38701
Converting text 38801
Converting text 38901
Converting text 39001
Converting text 39101
Converting text 39201
Converting text 39301
Converting text 39401
Converting text 39501
Converting text 39601
Converting text 39701
Converting text 39801
Converting text 39901
Converting text 40001
Converting text 40101
Converting text 40201
Converting text 40301
Converting text 40401
Converting text 40501
Converting text 40601
Converting text 40701
Converting text 40801
Converting text 40901
Converting text 41001
Converting text 41101
Converting text 41201
Converting text 41301
Converting text 41401
Converting text 41501
Converting text 41601
Converting text 41701
Converting text 41801
Converting text 41901
Converting text 42001
Converting text 42101
Converting text 42201
Converting

Converting text 75101
Converting text 75201
Converting text 75301
Converting text 75401
Converting text 75501
Converting text 75601
Converting text 75701
Converting text 75801
Converting text 75901
Converting text 76001
Converting text 76101
Converting text 76201
Converting text 76301
Converting text 76401
Converting text 76501
Converting text 76601
Converting text 76701
Converting text 76801
Converting text 76901
Converting text 77001
Converting text 77101
Converting text 77201
Converting text 77301
Converting text 77401
Converting text 77501
Converting text 77601
Converting text 77701
Converting text 77801
Converting text 77901
Converting text 78001
Converting text 78101
Converting text 78201
Converting text 78301
Converting text 78401
Converting text 78501
Converting text 78601
Converting text 78701
Converting text 78801
Converting text 78901
Converting text 79001
Converting text 79101
Converting text 79201
Converting text 79301
Converting text 79401
Converting text 79501
Converting

Converting text 111901
Converting text 112001
Converting text 112101
Converting text 112201
Converting text 112301
Converting text 112401
Converting text 112501
Converting text 112601
Converting text 112701
Converting text 112801
Converting text 112901
Converting text 113001
Converting text 113101
Converting text 113201
Converting text 113301
Converting text 113401
Converting text 113501
Converting text 113601
Converting text 113701
Converting text 113801
Converting text 113901
Converting text 114001
Converting text 114101
Converting text 114201
Converting text 114301
Converting text 114401
Converting text 114501
Converting text 114601
Converting text 114701
Converting text 114801
Converting text 114901
Converting text 115001
Converting text 115101
Converting text 115201
Converting text 115301
Converting text 115401
Converting text 115501
Converting text 115601
Converting text 115701
Converting text 115801
Converting text 115901
Converting text 116001
Converting text 116101
Converting 

Converting text 147601
Converting text 147701
Converting text 147801
Converting text 147901
Converting text 148001
Converting text 148101
Converting text 148201
Converting text 148301
Converting text 148401
Converting text 148501
Converting text 148601
Converting text 148701
Converting text 148801
Converting text 148901
Converting text 149001
Converting text 149101
Converting text 149201
Converting text 149301
Converting text 149401
Converting text 149501
Converting text 149601
Converting text 149701
Converting text 149801
Converting text 149901
Converting text 150001
Converting text 150101
Converting text 150201
Converting text 150301
Converting text 150401
Converting text 150501
Converting text 150601
Converting text 150701
Converting text 150801
Converting text 150901
Converting text 151001
Converting text 151101
Converting text 151201
Converting text 151301
Converting text 151401
Converting text 151501
Converting text 151601
Converting text 151701
Converting text 151801
Converting 

...

Converting text 118501
Converting text 118601
Converting text 118701
Converting text 118801
Converting text 118901
Converting text 119001
Converting text 119101
Converting text 119201
Converting text 119301
Converting text 119401
Converting text 119501
Converting text 119601
Converting text 119701
Converting text 119801
Converting text 119901
Converting text 120001
Converting text 120101
Converting text 120201
Converting text 120301
Converting text 120401
Converting text 120501
Converting text 120601
Converting text 120701
Converting text 120801
Converting text 120901
Converting text 121001
Converting text 121101
Converting text 121201
Converting text 121301
Converting text 121401
Converting text 121501
Converting text 121601
Converting text 121701
Converting text 121801
Converting text 121901
Converting text 122001
Converting text 122101
Converting text 122201
Converting text 122301
Converting text 122401
Converting text 122501
Converting text 122601
Converting text 122701
Converting 

Converting text 155701
Converting text 155801
Converting text 155901
Converting text 156001
Converting text 156101
Converting text 156201
Converting text 156301
Converting text 156401
Converting text 156501
Converting text 156601
Converting text 156701
Converting text 156801
Converting text 156901
Converting text 157001
Converting text 157101
Converting text 157201
Converting text 157301
Converting text 157401
Converting text 157501
Converting text 157601
Converting text 157701
Converting text 157801
Converting text 157901
Converting text 158001
Converting text 158101
Converting text 158201
Converting text 158301
Converting text 158401
Converting text 158501
Converting text 158601
Converting text 158701
Converting text 158801
Converting text 158901
Converting text 159001
Converting text 159101
Converting text 159201
Converting text 159301
Converting text 159401
Converting text 159501
Converting text 159601
Converting text 159701
Converting text 159801
Converting text 159901
Converting 

Converting text 191701
Converting text 191801
Converting text 191901
Converting text 192001
Converting text 192101
Converting text 192201
Converting text 192301
Converting text 192401
Converting text 192501
Converting text 192601
Converting text 192701
Converting text 192801
Converting text 192901
Converting text 193001
Converting text 193101
Converting text 193201
Converting text 193301
Converting text 193401
Converting text 193501
Converting text 193601
Converting text 193701
Converting text 193801
Converting text 193901
Converting text 194001
Converting text 194101
Converting text 194201
Converting text 194301
Converting text 194401
Converting text 194501
Converting text 194601
Converting text 194701
Converting text 194801
Converting text 194901
Converting text 195001
Converting text 195101
Converting text 195201
Converting text 195301
Converting text 195401
Converting text 195501
Converting text 195601
Converting text 195701
Converting text 195801
Converting text 195901
Converting 

Converting text 228001
Converting text 228101
Converting text 228201
Converting text 228301
Converting text 228401
Converting text 228501
Converting text 228601
Converting text 228701
Converting text 228801
Converting text 228901
Converting text 229001
Converting text 229101
Converting text 229201
Converting text 229301
Converting text 229401
Converting text 229501
Converting text 229601
Converting text 229701
Converting text 229801
Converting text 229901
Converting text 230001
Converting text 230101
Converting text 230201
Converting text 230301
Converting text 230401
Converting text 230501
Converting text 230601
Converting text 230701
Converting text 230801
Converting text 230901
Converting text 231001
Converting text 231101
Converting text 231201
Converting text 231301
Converting text 231401
Converting text 231501
Converting text 231601
Converting text 231701
Converting text 231801
Converting text 231901
Converting text 232001
Converting text 232101
Converting text 232201
Converting 

Converting text 265301
Converting text 265401
Converting text 265501
Converting text 265601
Converting text 265701
Converting text 265801
Converting text 265901
Converting text 266001
Converting text 266101
Converting text 266201
Converting text 266301
Converting text 266401
Converting text 266501
Converting text 266601
Converting text 266701
Converting text 266801
Converting text 266901
Converting text 267001
Converting text 267101
Converting text 267201
Converting text 267301
Converting text 267401
Converting text 267501
Converting text 267601
Converting text 267701
Converting text 267801
Converting text 267901
Converting text 268001
Converting text 268101
Converting text 268201
Converting text 268301
Converting text 268401
Converting text 268501
Converting text 268601
Converting text 268701
Converting text 268801
Converting text 268901
Converting text 269001
Converting text 269101
Converting text 269201
Converting text 269301
Converting text 269401
Converting text 269501
Converting 

In [8]:
dataset.head()

,Unnamed: 0,PRODUCT,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,...,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT,BENEFITS_FILLED,DRAWBACKS_FILLED,TEXT_FILLED,TEXT_CONVERTED,BENEFITS_CONVERTED,DRAWBACKS_CONVERTED
0,0,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Что особенно ценно - при профилактике принтера...,23.10.16 22:26:00,...,"Рядом ""на стол"" не поставишь, место много зани...",1.0,2.0,0.0,"хорошая скорость печати, двусторонняя печать, ...","Рядом ""на стол"" не поставишь, место много зани...",Что особенно ценно - при профилактике принтера...,что_CONJ особенно_ADVB ценно_ADJS при_PREP про...,хорошая_ADJF скорость_NOUN печати_NOUN двустор...,рядом_NOUN на_PREP стол_NOUN не_PRCL поставишь...
1,1,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Он у нас три месяца в офисе стоит уже - большо...,17.04.16 11:17:42,...,NaN,NaN,0.0,2.0,,,Он у нас три месяца в офисе стоит уже - большо...,он_NPRO у_PREP нас_NPRO три_NUMR месяца_NOUN в...,,
2,2,30024725,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,"Просто небо и земля с тем принтером, который у...",15.04.16 20:22:47,...,NaN,NaN,0.0,1.0,,,"Просто небо и земля с тем принтером, который у...",просто_PRCL небо_NOUN и_CONJ земля_NOUN с_PREP...,,
3,3,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,С такой вроде бы навороченной техникой все смо...,У нас в кабинете недавно его поставили. Первое...,18.04.16 20:01:57,...,NaN,NaN,0.0,0.0,,,У нас в кабинете недавно его поставили. Первое...,у_PREP нас_NPRO в_PREP кабинете_NOUN недавно_A...,,
4,4,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Очень многофункциональная штука. Есть все нео...,23.04.16 12:03:39,...,NaN,NaN,1.0,0.0,,,Очень многофункциональная штука. Есть все нео...,очень_ADVB многофункциональная_ADJF штука_NOUN...,,


In [9]:
dataset.to_csv("checkpoint/reviews_converted.csv", index=False)

# Этап 2. Отбор N-грам

Идея следующая - отобрать n-грамы, для которых значение tf-idf будет выше прочих n-грам текстов о этом товаре (такие n-грамы должны упоминаться относительно часто). После этого можно будет объединить n-грамы и построить по ним текст, но это - следующий этап

In [1]:
import pandas as pd
dataset = pd.read_csv("checkpoint/reviews_converted.csv")

Выберем наиболее часто встречаемый продукт (а также следующий за ним)

In [2]:
from scipy.stats import mode
import numpy as np

product = np.array(dataset["PRODUCT"])
top_product = mode(product)

In [3]:
top_product = top_product.mode[0]

In [4]:
top_product

10006894

In [5]:
top_product = 10006894

Далее применим только относящуюся к этому продукту часть датасета

In [6]:
subdataset_index = dataset["PRODUCT"] == top_product
subdataset = dataset.loc[subdataset_index, ["PRODUCT", "TEXT_CONVERTED", "BENEFITS_CONVERTED", "DRAWBACKS_CONVERTED"]]

In [7]:
subdataset_texts = list(map(str,
                            list(subdataset["TEXT_CONVERTED"]) + 
                            list(subdataset["BENEFITS_CONVERTED"]) + 
                            list(subdataset["DRAWBACKS_CONVERTED"])))

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

Построим несколько tf-idf (над отдельными "словами", биграммами и 3-граммами/4-граммами)

In [9]:
from itertools import chain

subdataset_sentences = []
for text in subdataset_texts:
    dot_del = text.split(".")
    for subtext in dot_del:
        subdataset_sentences += subtext.split(',')

In [10]:
print("Fitting TFIDF over 1-grams")
tfidf1 = TfidfVectorizer(ngram_range=(1,1)).fit(subdataset_sentences)
print("Fitting TFIDF over 2-grams")
tfidf2 = TfidfVectorizer(ngram_range=(2,2)).fit(subdataset_sentences)
print("Fitting TFIDF over 3-grams")
tfidf3 = TfidfVectorizer(ngram_range=(3,3)).fit(subdataset_sentences)
print("Fitting TFIDF over 4-grams")
tfidf4 = TfidfVectorizer(ngram_range=(4,4)).fit(subdataset_sentences)

Fitting TFIDF over 1-grams
Fitting TFIDF over 2-grams
Fitting TFIDF over 3-grams
Fitting TFIDF over 4-grams


In [11]:
top_product_reviews = subdataset.loc[subdataset["PRODUCT"] == top_product]

Определим функцию, принимающую на вход полученную от tfidf матрицу и tfidf и возвращающую имена фич tfidf, отсортированные по среднему (на этих текстах) значению фичи

In [12]:
def tfidf_order_features(tfidf, matrix):
    mean_features = np.asarray(matrix.mean(axis=0))[0]
    ordered_features = mean_features.argsort()[::-1]
    feature_names = tfidf.get_feature_names()
    result = []
    for feature in ordered_features:
        result.append(feature_names[feature])
    return np.array(result)

In [13]:
top_product_tfidf1 = tfidf1.transform(subdataset_sentences)
top_product_tfidf2 = tfidf2.transform(subdataset_sentences)
top_product_tfidf3 = tfidf3.transform(subdataset_sentences)
top_product_tfidf4 = tfidf4.transform(subdataset_sentences)

In [14]:
tfidf_order_features(tfidf1, top_product_tfidf1)[:10]

array(['nan', 'в_prep', 'не_prcl', 'и_conj', 'на_prep', 'нет_pred',
       'большой_adjf', 'телевизор_noun', 'что_conj', 'очень_advb'],
      dtype='<U27')

In [15]:
tfidf_order_features(tfidf2, top_product_tfidf2)[:20]

array(['их_npro нет_pred', 'этот_adjf телевизор_noun',
       'большой_adjf экран_noun', 'очень_advb большой_adjf',
       'и_conj не_prcl', 'дорогая_adjf доставка_noun', 'у_prep меня_npro',
       'доступная_adjf цена_noun', 'пришлось_verb продать_infn',
       'а_conj так_conj', 'за_prep такие_adjf', 'такие_adjf деньги_noun',
       'хорошее_adjf качество_noun', 'на_prep нем_adjs',
       'очень_advb дорого_advb', 'всем_adjf рекомендую_verb',
       'у_prep нас_npro', 'не_prcl обнаружил_verb', 'на_prep стену_noun',
       'так_conj как_conj'],
      dtype='<U42')

In [16]:
tfidf_order_features(tfidf3, top_product_tfidf3)[:30]

array(['за_prep такие_adjf деньги_noun',
       'очень_advb доступный_adjf телек_noun',
       'за_prep такую_adjf цену_noun', 'большой_adjf и_conj цветной_adjf',
       'не_prcl поместился_verb в_prep',
       'остался_verb без_prep квартиры_noun',
       'цветной_adjf есть_intj пульт_noun',
       'очень_advb хороший_adjf телевизор_noun',
       'судя_grnd по_prep отзывам_noun', 'за_prep свои_adjf деньги_noun',
       'цветной_adjf и_conj большой_adjf',
       'сделан_prts в_prep россии_noun',
       'вписался_verb в_prep интерьер_noun',
       'хорошее_adjf качество_noun картинки_noun',
       'у_prep нас_npro в_prep', 'купить_infn этот_adjf телевизор_noun',
       'не_prcl могу_verb настроить_infn',
       'телевизор_noun за_prep свои_adjf', 'не_prcl знаю_verb что_conj',
       'продал_verb квартиру_noun купил_verb',
       'цена_noun это_prcl его_npro', 'это_prcl его_npro основной_adjf',
       'страна_noun производитель_noun россия_noun',
       'в_prep ипотеку_noun на_prep', 'кн

In [17]:
tfidf_order_features(tfidf4, top_product_tfidf4)[:40]

array(['цена_noun это_prcl его_npro основной_adjf',
       'телевизор_noun за_prep свои_adjf деньги_noun',
       'удобно_advb лежит_verb в_prep руке_noun',
       'на_prep завтраках_noun в_prep школе_noun',
       'в_prep описании_noun одни_adjf плюсы_noun',
       'дешёвый_adjf могут_verb купить_infn все_adjf',
       'тем_noun более_advb стоит_verb копейки_noun',
       'я_npro думал_verb телевизор_noun водонепроницаем_adjs',
       'батарейки_noun от_prep пульта_noun дорого_advb',
       'теперь_advb придётся_verb открывать_infn кинотеатр_noun',
       'мало_advb почек_noun для_prep оплаты_noun',
       'работает_verb от_prep сети_noun вольт_noun',
       'станции_noun в_prep форточку_noun протянули_verb',
       'она_noun оторваться_infn не_prcl может',
       'ловит_verb бесплатные_adjf цифровые_adjf каналы_noun',
       'демократичная_adjf цена_noun неплохое_adjf качество_noun',
       'пришлось_verb продать_infn все_adjf золото_noun',
       'в_prep целом_noun очень_advb хороши

Видим, что с 3-граммами и 4-граммами видны признаки, из которых можно извлечь тексты (но необходимо будет объединить многие признаки и ввести фильтрацию).

In [18]:
import re

ignorance_filter = lambda text: bool(re.match(".*prep", text)) or \
    bool(re.match(".*infn", text)) or \
    bool(re.match(".*verb", text))
feature_filter = lambda text: bool(re.match(".*adjf .*noun", text)) and not ignorance_filter(text)

In [19]:
list(filter(feature_filter, tfidf_order_features(tfidf3, top_product_tfidf3)))[:20]

['очень_advb доступный_adjf телек_noun',
 'цветной_adjf есть_intj пульт_noun',
 'очень_advb хороший_adjf телевизор_noun',
 'хорошее_adjf качество_noun картинки_noun',
 'кривые_noun задние_adjf ножки_noun',
 'реально_adjs крутое_adjf изображение_noun',
 'хоршая_adjf картинка_noun изогнутый_adjf',
 'нет_pred оранжевого_adjf корпуса_noun',
 'большой_adjf чёткий_adjf экран_noun',
 'дешевле_comp некоторых_adjf квартир_noun',
 'хорошее_adjf качество_noun сборки_noun',
 'хороший_adjf бюджетный_adjf телевизор_noun',
 'нет_pred настенного_adjf крепления_noun',
 'отличное_adjf антикризисное_adjf предложение_noun',
 'слишком_advb дорогая_adjf доставка_noun',
 'слишком_advb маленький_adjf размер_noun',
 'супер_adjf хороший_adjf телевизер_noun',
 'этот_adjf тв_noun и_conj',
 'не_prcl плохой_adjf телик_noun',
 'такую_adjf цену_noun нет_pred']

In [20]:
import pickle as pcl

with open("checkpoint/tfidf2.pkl", "wb") as target:
    pcl.dump(tfidf2, target)
with open("checkpoint/tfidf3.pkl", "wb") as target:
    pcl.dump(tfidf3, target)
with open("checkpoint/tfidf4.pkl", "wb") as target:
    pcl.dump(tfidf4, target)

In [21]:
top_features_tfidf2 = np.array(list(
    filter(feature_filter, tfidf_order_features(tfidf2, top_product_tfidf2)))[:50]
)
np.save("checkpoint/top-tfidf2.npy", top_features_tfidf2)
top_features_tfidf3 = np.array(list(
    filter(feature_filter, tfidf_order_features(tfidf3, top_product_tfidf3)))[:50]
)
np.save("checkpoint/top-tfidf3.npy", top_features_tfidf3)
top_features_tfidf4 = np.array(list(
    filter(feature_filter, tfidf_order_features(tfidf4, top_product_tfidf4)))[:50]
)
np.save("checkpoint/top-tfidf4.npy", top_features_tfidf4)

# Кластеризация и отбор N-грам

Для объединения N-грам применим word mover distance (предварительно обучив word2vec, т.к. использование предобученной модели от rusvectores потребует, как минимум, адаптации)

In [1]:
import pandas as pd
dataset = pd.read_csv("checkpoint/reviews_converted.csv")

all_tokens = list(map(lambda text: str(text).lower().split(" "),
                      list(dataset["TEXT_CONVERTED"]) + 
                      list(dataset["BENEFITS_CONVERTED"]) + 
                      list(dataset["DRAWBACKS_CONVERTED"])))

In [2]:
from gensim.models import Word2Vec

word2vec = Word2Vec(all_tokens)

In [3]:
word2vec.wv.save_word2vec_format('checkpoint/word2vec.bin', binary=True)

In [4]:
from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format('checkpoint/word2vec.bin', binary=True)

In [5]:
from scipy.spatial.distance import cosine

def distance(doc1, doc2, word2vec):
    doc1vec = np.array([np.zeros([word2vec.vector_size])] + 
                       [word2vec[token] for token in doc1.split(" ") if token in word2vec]).sum(axis=0)
    doc2vec = np.array([np.zeros([word2vec.vector_size])] + 
                       [word2vec[token] for token in doc2.split(" ") if token in word2vec]).sum(axis=0)
    diff = doc1vec - doc2vec
    return cosine(doc1vec, doc2vec)

In [8]:
import pickle

with open("checkpoint/tfidf2.pkl", "rb") as src:
    tfidf2 = pickle.load(src)
with open("checkpoint/tfidf3.pkl", "rb") as src:
    tfidf3 = pickle.load(src)
with open("checkpoint/tfidf4.pkl", "rb") as src:
    tfidf4 = pickle.load(src)

In [10]:
import numpy as np

top_product_tfidf2 = np.load("checkpoint/top-tfidf2.npy")
top_product_tfidf3 = np.load("checkpoint/top-tfidf3.npy")
top_product_tfidf4 = np.load("checkpoint/top-tfidf4.npy")

In [11]:
top_features = list(set(top_product_tfidf2.tolist() + 
                        top_product_tfidf3.tolist() + 
                        top_product_tfidf4.tolist()))
top_features.sort()

In [12]:
distances = np.zeros([len(top_features), len(top_features)])
for i, feature1 in enumerate(top_features):
    for j, feature2 in enumerate(top_features):
        features_distance = distance(feature1, feature2, word2vec)
        distances[i, j] = features_distance
        distances[j, i] = features_distance

In [13]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(0.2, min_samples=1, metric="precomputed")
clusters = dbscan.fit_predict(distances)

items = {}
for cluster, feature in zip(clusters, top_features):
    items[cluster] = items.get(cluster, []) + [feature]

In [14]:
choosen_ngrams = []
for key, values in items.items():
    values_indices = np.array([top_features.index(val) for val in values])
    if len(values) < 2:
        continue
    values_distances = distances[values_indices, :][:, values_indices]
    index_mean_distances = np.zeros([len(values_indices)])
    for i in range(0, len(values_indices)):
        index_mean_distances[i] = np.delete(values_distances[i], i, axis=0).mean()
    choosen_ngram = values[index_mean_distances.argmin()]
    print(key, choosen_ngram)
    choosen_ngrams.append(choosen_ngram)

1 хорошее_adjf качество_noun картинки_noun
5 большой_adjf всю_adjf жизнь_noun
6 большой_adjf экран_noun цветное_adjf изображение_noun
8 отличный_adjf телевизор_noun крутая_adjf картинка_noun
16 громкий_adjf звук_noun
17 да_prcl не_prcl плохой_adjf телик_noun
18 этот_adjf телевизор_noun
20 доступная_adjf цена_noun
24 отображение_noun каждого_adjf цвета_noun и_conj
26 и_conj большой_adjf телек_noun норм_noun
28 супер_adjf качество_noun удобная_adjf
29 тяжёлый_adjf кг_noun пластмассового_adjf веса_noun
33 моё_adjf лицо_noun немного_advb
40 одни_adjf плюсы_noun
42 отличное_adjf антикризисное_adjf предложение_noun
52 эти_adjf деньги_noun
62 такую_adjf цену_noun их_npro нет_pred
66 упаковочная_adjf коробка_noun не_prcl
69 цветной_adjf есть_intj пульт_noun


Видим, что из собранных N-грам, похоже, можно собрать тексты. Но некоторые из них потребуется удалить или доработать.

In [15]:
np.save("checkpoint/chosen_ngrams.npy", np.array(choosen_ngrams))

# Сборка текста

Запишем правила сборки итогового текста (воспользовавшись работающими с pos-тегами регулярными выражениями для правил расстановки препинания/удаления pos-тегов/обрезки текста)

In [17]:
import re

rules = [lambda text: re.sub("_adjf+ (\w+)_intj", ", \g<1>", text),
         lambda text: re.sub("_noun+ (\w+)_adjf", ", \g<1>", text),
         lambda text: re.sub("^\w+_conj", "", text),
         lambda text: re.sub("\w+_conj$", "", text),
         lambda text: re.sub("^\w+_pred", "", text),
         lambda text: re.sub("\w+_pred$", "", text),
         lambda text: re.sub("^\w+_precl", "", text),
         lambda text: re.sub("\w+_precl$", "", text),
         lambda text: re.sub("_[a-z]+", "", text),
         lambda text: text.strip()]

In [19]:
def apply_rules(rules, text):
    if isinstance(text, list):
        return [apply_rules(rules, item) for item in text]
    for rule in rules:
        text = rule(text)
    return text

In [25]:
choosen_ngrams = np.load("checkpoint/chosen_ngrams.npy")

apply_rules(rules, choosen_ngrams.tolist())

['хорошее качество картинки',
 'большой всю жизнь',
 'большой экран, цветное изображение',
 'отличный телевизор, крутая картинка',
 'громкий звук',
 'да не плохой телик',
 'этот телевизор',
 'доступная цена',
 'отображение, каждого цвета',
 'большой телек норм',
 'супер качество, удобная',
 'тяжёлый кг, пластмассового веса',
 'моё лицо немного',
 'одни плюсы',
 'отличное антикризисное предложение',
 'эти деньги',
 'такую цену их',
 'упаковочная коробка не',
 'цветной, есть пульт']

Видим, что пункты образованы, но далеки от идеала.

Возможно - стоит добавить n-грамы большей размерности и фильтровать их с помощью не только внутрикластерных расстояний, но и языковых моделей. Это, вероятно, убрало бы некоторые явно неверно обработанные результаты.